In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
import talib as ta
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam

# Reading CSV file

In [ ]:
df = pd.read_excel('BTC-USD.xlsx' , index_col = 'Date')

In [ ]:
df

In [ ]:
#df = df.drop( ['<TICKER>' ,  '<OPENINT>' , '<OPENINT>.1'  ] , axis = 1)

In [ ]:
values = df.values

In [ ]:
df = df.dropna()

# Preprocessing
## Finding all 0 values which are the sign of closed days and turning them to numpy nan 


In [ ]:
df = df[df>0]

In [ ]:
df

In [ ]:
groups = [0,1,2,3,4]

In [ ]:
i = 1
pyplot.figure(figsize = (20,10))
for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(values[:, group])
    pyplot.title(df.columns[group], y=0.5, loc='right')
    i += 1
pyplot.show()

## Interpolating the closed days of the stock by linear style

In [ ]:
df['Open'].interpolate(method = 'linear' , inplace = True)
df['High'].interpolate(method = 'linear' , inplace = True)
df['Low'].interpolate(method = 'linear' , inplace = True)
df['Volume'].interpolate(method = 'linear' , inplace = True)
df['Close'].interpolate(method = 'linear' , inplace = True)

In [ ]:
a = df['Open']
b = df['High']
c = df['Low']
d = df['Volume']
e = df['Close']

In [ ]:
df_new = pd.DataFrame({'Date':df.index, 'Open':a ,'High':b ,'Low':c , 'Close':e ,'Volume': d })
df_new.set_index(['Date'] , inplace = True)

In [ ]:
values_1 = df_new.values

In [ ]:
i = 1
pyplot.figure(figsize = (20,10))
for group in groups:
    pyplot.subplot(len(groups), 1, i)
    pyplot.plot(values_1[:, group])
    pyplot.title(df_new.columns[group], y=0.5, loc='right')
    i += 1
pyplot.show()

# Making the continuing code compatible

In [ ]:
df = df_new

# Generating Indicators on the stock (Feature creation)

In [ ]:
# ADX - Average Directional Movement Index
adx = ta.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

# ADXR - Average Directional Movement Index Rating
adxr = ta.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)

# APO - Absolute Price Oscillator
apo = ta.APO(df['Close'], fastperiod=12, slowperiod=26, matype=0)

# AROON - Aroon
aroondown, aroonup = ta.AROON(df['High'], df['Low'], timeperiod=14)

# AROONOSC - Aroon Oscillator
AROONOSC = ta.AROONOSC(df['High'], df['Low'], timeperiod=14)

# BOP - Balance Of Power
bop = ta.BOP(df['Open'],df['High'], df['Low'],df['Close'])

# CCI - Commodity Channel Index
cci = ta.CCI(df['High'], df['Low'], df['Close'], timeperiod=14)

# CMO - Chande Momentum Oscillator
cmo = ta.CMO(df['Close'], timeperiod=14)

# DX - Directional Movement Index
dx = ta.DX(df['High'], df['Low'], df['Close'], timeperiod=14)

# MACD - Moving Average Convergence/Divergence
macd, macdsignal, macdhist = ta.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

# MFI - Money Flow Index
mfi = ta.MFI(df['High'],df['Low'], df['Close'], df['Volume'], timeperiod=14)

# MINUS_DI - Minus Directional Indicator
minus_di = ta.MINUS_DI(df['High'],df['Low'], df['Close'], timeperiod=14)

# MINUS_DM - Minus Directional Movement
minus_dm = ta.MINUS_DM(df['High'], df['Low'], timeperiod=14)

# MOM - Momentum
momentum = ta.MOM(df['Close'], timeperiod=10)

# PLUS_DI - Plus Directional Indicator
plus_di = ta.PLUS_DI(df['High'], df['Low'], df['Close'], timeperiod=14)

# PLUS_DM - Plus Directional Movement
plus_dm = ta.PLUS_DM(df['High'], df['Low'], timeperiod=14)

# PPO - Percentage Price Oscillator
ppo = ta.PPO(df['Close'], fastperiod=12, slowperiod=26, matype=0)

# ROC - Rate of change : ((price/prevPrice)-1)*100
roc = ta.ROC(df['Close'], timeperiod=10)

# ROCP - Rate of change Percentage: (price-prevPrice)/prevPrice
rocp = ta.ROCP(df['Close'], timeperiod=10)

# ROCR - Rate of change ratio: (price/prevPrice)
rocr = ta.ROCR(df['Close'], timeperiod=10)

# ROCR100 - Rate of change ratio 100 scale: (price/prevPrice)*100
rocr100 = ta.ROCR100(df['Close'], timeperiod=10)

# RSI - Relative Strength Index
rsi = ta.RSI(df['Close'], timeperiod=14)

#STOCH - Stochastic
slowk, slowd = ta.STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

# STOCHF - Stochastic Fast
fastk, fastd = ta.STOCHF(df['High'], df['Low'], df['Close'], fastk_period=5, fastd_period=3, fastd_matype=0)

# STOCHRSI - Stochastic Relative Strength Index
fastkrsi, fastdrsi = ta.STOCHRSI(df['Close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)

# TRIX - 1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
trix = ta.TRIX(df['Close'], timeperiod=30)

# ULTOSC - Ultimate Oscillator
ultosc = ta.ULTOSC(df['High'], df['Low'], df['Close'], timeperiod1=7, timeperiod2=14, timeperiod3=28)

# WILLR - Williams' %R
willr = ta.WILLR(df['High'], df['Low'],  df['Close'], timeperiod=14)

# Indicator DataFrame

In [ ]:
indicator = pd.DataFrame({ 'ADX':adx , 'ADXR':adxr , 'APO' :apo , 'AROON_DOWN': aroondown ,'AROON_UP':aroonup , 'AROONOSC' :AROONOSC,
                          'BOP':bop , 'CCI':cci ,'CMO':cmo ,'DX': dx , 'MACD': macd ,'MFI':mfi , 'MINUS_DI': minus_di , 'MINUS_DM':minus_dm, 
                          'MOM':momentum ,'PLUS_DI':plus_di,'PLUS_DM': plus_dm , 'PPO':ppo , 'ROC':roc , 'ROCP':rocp , 'ROCR':rocr ,
                         'ROCR100' : rocr100 , 'RSI':rsi ,'STOCK_K':slowk , 'STOCK_D':slowd ,'STOCKF_K':fastk , 'STOCKF_D':fastd,
                          'STOCHRSI_k':fastkrsi ,'STOCHRSI_D':fastdrsi,  'TRIX':trix , 'ULTOSC':ultosc , 'WILLR':willr })

In [ ]:
indicator

In [ ]:
indicator = indicator.dropna()

In [ ]:
indicator

# creating lag of close price

In [ ]:
close_price = df['Close']

In [ ]:
close_price_lag1 = close_price.shift()
close_price_lag2 = close_price.shift(2)
close_price_lag3 = close_price.shift(3)
close_price_lag4 = close_price.shift(4)
close_price_lag5 = close_price.shift(5)
close_price_lag6 = close_price.shift(6)
close_price_lag7 = close_price.shift(7)
close_price_lag8 = close_price.shift(8)
close_price_lag9 = close_price.shift(9)
close_price_lag10 = close_price.shift(10)

In [ ]:
close_price_including_lags = pd.DataFrame({'Close':close_price ,'Close LAG1':close_price_lag1 ,'Close LAG2':close_price_lag2
                                          ,'Close LAG3':close_price_lag3 ,'Close LAG4':close_price_lag4 ,'Close LAG5':close_price_lag5
                                          ,'Close LAG6':close_price_lag6 ,'Close LAG7':close_price_lag7 ,'Close LAG8':close_price_lag8
                                          ,'Close LAG9':close_price_lag9 ,'Close LAG10':close_price_lag10})

In [ ]:
close_price_including_lags = close_price_including_lags[88:]

In [ ]:
close_price_including_lags

# FINAL DATA FRAME INCLUDING TARGET AND FEATURES
# (before feature selection)


In [ ]:
final_data = pd.concat([df[88:] , indicator] , axis = 1)

In [ ]:
final_data = final_data.drop('Close' , axis = 1)

In [ ]:
final_data = pd.concat([close_price_including_lags , final_data] , axis = 1)

In [ ]:
final_data = final_data.dropna()

In [ ]:
final_data = final_data.reset_index()

In [ ]:
final_data = final_data.drop(['Date'] , axis = 1)

In [ ]:
final_data

# Feature correlation analysis (to see which feature selection suits better)
## Random forest vs RFE(linear)
### if correlation is low : Rnadom forest and if correlation is high : RFE(linear)

In [ ]:
corr_matrix_features = final_data.corr()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(corr_matrix_features , cmap='coolwarm')

# Feature Selection

#### As the clasifier can only get integers we can multiply all values in 100 to not miss any differnces and boundaries and then convert them into integers to be accepted in the random forrest classification

In [ ]:
X_feature_selection = final_data.drop(['Close'] , axis=1 )
y_feature_selection = final_data['Close']

X_feature_selection = (X_feature_selection*100).astype(int)
y_feature_selection = (y_feature_selection*100).astype(int)

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_feature_selection, y_feature_selection)

In [ ]:
sel.get_support()

In [ ]:
selected_features= X_feature_selection.columns[(sel.get_support())]
number_of_selected_features = len(selected_features)
print(number_of_selected_features , 'features have beed choosed through RandomForest Clasifier')

In [ ]:
selected_features = selected_features.tolist()

# FINAL DATA FRAME INCLUDING TARGET AND FEATURES
# (after feature selection)

In [ ]:
close_df = pd.DataFrame({ 'Close' : final_data['Close'] })
selected_features_dataframe = final_data[selected_features]
final_data = pd.concat([close_df , selected_features_dataframe] , axis =1)

In [ ]:
final_data

#                      Train Test split 
##                           The sequence is divided into 3 small sequences
###                                   (Each small sequence has its own training and test set)

In [ ]:
#lenght of the whole sequemce

seq_len = len(final_data)

In [ ]:
#lenght of each small sequences

small_seq_len = seq_len / 3
small_seq_len = int(small_seq_len)

## train test split for the first sequence
### normalisation (fitted on train set and transformed to train and test set)

In [ ]:
normalizer = MinMaxScaler()

In [ ]:
train_data_first_seq = final_data[ : int(small_seq_len*0.9)]

In [ ]:
# test calculation
normalizer.fit(train_data_first_seq)

In [ ]:
train_data_first_seq = normalizer.transform(train_data_first_seq)

In [ ]:
test_data_first_seq = final_data[int(small_seq_len*0.9) : int((small_seq_len*0.9) + small_seq_len*0.1)]

In [ ]:
test_data_first_seq = normalizer.transform(test_data_first_seq)

## train test split for the second sequence
### normalisation (fitted on train set and transformed to train and test set)

In [ ]:
train_data_second_seq = final_data[int((small_seq_len*0.9) + (small_seq_len*0.1)) : int((small_seq_len*0.9) + (small_seq_len*0.1) + 
                                                                                     (small_seq_len*0.9)) ]

In [ ]:
normalizer.fit(train_data_second_seq)

In [ ]:
train_data_second_seq = normalizer.transform(train_data_second_seq)

In [ ]:
test_data_second_seq = final_data[int((small_seq_len*0.9) + (small_seq_len*0.1) + (small_seq_len*0.9)) : int((small_seq_len*0.9) + (small_seq_len*0.1) + 
                                                                                     (small_seq_len*0.9) + (small_seq_len*0.1)) ]

In [ ]:
test_data_second_seq = normalizer.transform(test_data_second_seq)

## train test split for the third sequence 
### normalisation (fitted on train set and transformed to train and test set)

In [ ]:
train_data_third_seq = final_data[int((small_seq_len*0.9) + (small_seq_len*0.1) +(small_seq_len*0.9) + (small_seq_len*0.1)) :
                                 int((small_seq_len*0.9) + (small_seq_len*0.1) +(small_seq_len*0.9) + (small_seq_len*0.1) +
                                    (small_seq_len*0.9))]

In [ ]:
normalizer.fit(train_data_third_seq)

In [ ]:
train_data_third_seq = normalizer.transform(train_data_third_seq)

In [ ]:
test_data_third_seq = final_data[int((small_seq_len*0.9) + (small_seq_len*0.1) +(small_seq_len*0.9) + (small_seq_len*0.1) + (small_seq_len*0.9)):]

In [ ]:
test_data_third_seq = normalizer.transform(test_data_third_seq)

# Runing the first sequence of the model

## Split into feature and target ( for the first sequence )


In [ ]:
# Convert Into numpy array
train_values = train_data_first_seq
test_values = test_data_first_seq

In [ ]:
train_X, train_y = train_values[:, 1:], train_values[:, 0]
test_X, test_y = test_values[:, 1:], test_values[:, 0]

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
train_y = train_y.reshape(630,1)
test_y = test_y.reshape(70,1)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# Before starting the model 
## I created a metric to calculate R^2
### it is created as followes

In [ ]:
def R_2(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# Training the Model

In [ ]:
from keras import backend as k
K.clear_session()

model = Sequential()

opt = Adam(lr = 1e-6 * 10**(110 / 40) , decay = 0.007) #  , clipnorm=1.0 
model.add(LSTM(30 , return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(120, return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(30))
model.add(Dropout(0.2))

model.add(Dense(1))

In [ ]:
model.compile(optimizer = opt , loss='mae' , metrics = ['mse' ] )
# fit network
history = model.fit(train_X, train_y, epochs = 1000, batch_size=64, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(18,6))

axes[0].plot(history.history['loss'], label='train')
axes[0].plot(history.history['val_loss'], label='test')
axes[0].legend()
axes[0].set_title('MAE loss')

axes[1].plot(history.history['mse'], label='train')
axes[1].plot(history.history['val_mse'], label='test')
axes[1].legend()
axes[1].set_title('MSE loss')


In [ ]:
plt.figure(figsize=[14 ,7] , )
plt.plot(history.history['R_2'], label='train')
plt.plot(history.history['val_R_2'], label='test')
plt.legend()
plt.title('R^2')

## Evaluating the model ( for the first sequence )

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.figure(figsize=(18,9))
plt.plot(yhat.flatten() , 'g-', marker = 'x' , markersize = 5  , label = 'Y_hat')
plt.plot(test_y , 'b-' , label = 'Target' )
plt.legend()
plt.title('F I T')
plt.xlabel('Time')
plt.ylabel('Close Price')

# Runing the second sequence of the model

## Split into feature and target ( for the second sequence )

In [ ]:
# Convert Into numpy array
train_values = train_data_second_seq
test_values = test_data_second_seq

In [ ]:
train_X, train_y = train_values[:, 1:], train_values[:, 0]
test_X, test_y = test_values[:, 1:], test_values[:, 0]

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# Training the model ( for the second sequence )

In [ ]:
from keras import backend as k
K.clear_session()

model = Sequential()

opt = Adam(lr = 1e-6 * 10**(110 / 40) , decay = 0.007) #  , clipnorm=1.0 
model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]) , return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(120, return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(30))
model.add(Dropout(0.2))

model.add(Dense(1))
model.compile(optimizer = opt , loss='mae' , metrics = ['mse' , R_2 ] )
# fit network
history = model.fit(train_X, train_y, epochs = 1000, batch_size=64, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(18,6))

axes[0].plot(history.history['loss'], label='train')
axes[0].plot(history.history['val_loss'], label='test')
axes[0].legend()
axes[0].set_title('MAE loss')

axes[1].plot(history.history['mse'], label='train')
axes[1].plot(history.history['val_mse'], label='test')
axes[1].legend()
axes[1].set_title('MSE loss')

In [ ]:
plt.figure(figsize=[14 ,7] , )
plt.plot(history.history['R_2'], label='train')
plt.plot(history.history['val_R_2'], label='test')
plt.legend()
plt.title('R^2')

## Evaluating the model ( for the second sequence )

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.figure(figsize=(18,9))
plt.plot(yhat.flatten() , 'g-', marker = 'x' , markersize = 5  , label = 'Y_hat')
plt.plot(test_y , 'b-' , label = 'Target' )
plt.legend()
plt.title('F I T')
plt.xlabel('Time')
plt.ylabel('Close Price')

# Runing the third sequence of the model

## Split into feature and target ( for the third sequence )

In [ ]:
# Convert Into numpy array
train_values = train_data_third_seq
test_values = test_data_third_seq

In [ ]:
train_X, train_y = train_values[:, 1:], train_values[:, 0]
test_X, test_y = test_values[:, 1:], test_values[:, 0]

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# Training the model ( for the third sequence )

In [ ]:
from keras import backend as k
K.clear_session()

model = Sequential()

opt = Adam(lr = 1e-6 * 10**(110 / 40) , decay = 0.007) #  , clipnorm=1.0 
model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]) , return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(120, return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(30))
model.add(Dropout(0.2))

model.add(Dense(1))
model.compile(optimizer = opt , loss='mae' , metrics = ['mse' , R_2 ] )
# fit network
history = model.fit(train_X, train_y, epochs = 1000, batch_size=64, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(18,6))

axes[0].plot(history.history['loss'], label='train')
axes[0].plot(history.history['val_loss'], label='test')
axes[0].legend()
axes[0].set_title('MAE loss')

axes[1].plot(history.history['mse'], label='train')
axes[1].plot(history.history['val_mse'], label='test')
axes[1].legend()
axes[1].set_title('MSE loss')

In [ ]:
plt.figure(figsize=[14 ,7] , )
plt.plot(history.history['R_2'], label='train')
plt.plot(history.history['val_R_2'], label='test')
plt.legend()
plt.title('R^2')

## Evaluating the model ( for the third sequence )

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.figure(figsize=(18,9))
plt.plot(yhat.flatten() , 'g-', marker = 'x' , markersize = 5  , label = 'Y_hat')
plt.plot(test_y , 'b-' , label = 'Target' )
plt.legend()
plt.title('F I T')
plt.xlabel('Time')
plt.ylabel('Close Price')

# Training the whole 10 year in one model and testing

## Split into feature and target

In [ ]:
#lenght of the whole sequemce
seq_len = len(final_data)
train_split = int(0.9*seq_len)
train_data_whole_seq = final_data[:train_split]
normalizer.fit(train_data_whole_seq)
train_data_whole_seq = normalizer.transform(train_data_whole_seq)
test_data_whole_seq = final_data[train_split:]
test_data_whole_seq = normalizer.transform(test_data_whole_seq)

In [ ]:
# Convert Into numpy array
train_values = train_data_whole_seq
test_values = test_data_whole_seq

In [ ]:
train_X, train_y = train_values[:, 1:], train_values[:, 0]
test_X, test_y = test_values[:, 1:], test_values[:, 0]

In [ ]:
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# Training the model for the whole sequence

In [ ]:
from keras import backend as k
K.clear_session()

model = Sequential()

opt = Adam(lr = 1e-6 * 10**(110 / 40) , decay = 0.007) #  , clipnorm=1.0 
model.add(LSTM(30, input_shape=(train_X.shape[1], train_X.shape[2]) , return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(110, return_sequences = True))
model.add(Dropout(0.05))

model.add(LSTM(20))
model.add(Dropout(0.1))

model.add(Dense(1))
model.compile(optimizer = opt , loss='mae' , metrics = ['mse' , R_2 ] )
# fit network
history = model.fit(train_X, train_y, epochs = 1000, batch_size=64, validation_data=(test_X, test_y), verbose=2, shuffle=False)

In [ ]:
# plot history
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(18,6))

axes[0].plot(history.history['loss'], label='train')
axes[0].plot(history.history['val_loss'], label='test')
axes[0].legend()
axes[0].set_title('MAE loss')

axes[1].plot(history.history['mse'], label='train')
axes[1].plot(history.history['val_mse'], label='test')
axes[1].legend()
axes[1].set_title('MSE loss')
#plt.savefig('gru train loggg' , dpi = 1000)

In [ ]:
plt.figure(figsize=[14 ,7] , )
plt.plot(history.history['R_2'], label='train')
plt.plot(history.history['val_R_2'], label='test')
plt.legend()
plt.title('R^2')

## Evaluating the model ( for the whole sequence)

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
plt.figure(figsize=(18,9))
plt.plot(yhat.flatten()+0.3 , 'g-', marker = 'x' , markersize = 5  , label = 'Y_hat')
plt.plot(test_y  , 'b-' , label = 'Target' )
plt.legend()
plt.title('F I T')
plt.xlabel('Time')
plt.ylabel('Close Price')

In [ ]:
#lstm = yhat.flatten() + 0.3
#gru = yhat.flatten() + 0.3

In [ ]:
zz = pd.DataFrame({'LSTM prediction':lstm , 'GRU prediction':gru , 'Date': df.index[-211:]} )
zz = zz.set_index ('Date')
zz['Target value'] = test_y
zz

In [ ]:

from matplotlib import style
style.use('seaborn')
plt.figure(figsize=(18,9))
plt.plot(zz['Target value'], 'b-'  , label = 'Target')
plt.plot(zz['LSTM prediction'] , 'g-*'  , markersize = 8 , label = 'LSTM prediction' )
plt.plot(zz['GRU prediction'] , 'r-*'  , markersize = 8 , label = 'GRU prediction' )
plt.legend()
plt.title('F I T')
plt.xlabel('Time')
plt.ylabel('Close Price scaled')
#plt.savefig('fig plot gru lstm bitcoin' , dpi = 1500)

In [ ]:
#zz.to_csv('fit plot lstm gru bitcoin.csv')